In [1]:
import torch
import os
import pandas as pd
import time
import logging
import transformers
from transformers import (
    GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
)
from torch.utils.data import Dataset, DataLoader

/home/adma224/anaconda3/envs/ml-dev/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ✅ Set device ONCE
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Memory optimizations (move these up)
torch.backends.cudnn.benchmark = False
torch.cuda.set_per_process_memory_fraction(0.6, device=0)
torch.cuda.empty_cache()

# Limit GPU & CPU usage
os.environ["OMP_NUM_THREADS"] = "4"  # Limit CPU threads
os.environ["MKL_NUM_THREADS"] = "4"

In [3]:
# Load the cleaned dataset
df = pd.read_csv("../data/cleaned_dataset.csv", index_col=0)

In [4]:
df

,headline
0,Over 4 Million Americans Roll Up Sleeves For O...
1,"American Airlines Flyer Charged, Banned For Li..."
2,23 Of The Funniest Tweets About Cats And Dogs ...
3,The Funniest Tweets From Parents This Week (Se...
4,Woman Who Called Cops On Black Bird-Watcher Lo...
...,...
209522,RIM CEO Thorsten Heins' 'Significant' Plans Fo...
209523,Maria Sharapova Stunned By Victoria Azarenka I...
209524,"Giants Over Patriots, Jets Over Colts Among M..."
209525,Aldon Smith Arrested: 49ers Linebacker Busted ...


In [5]:
df = df.dropna(subset=["headline"]).copy()  # Remove rows where 'headline' is NaN


In [6]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 doesn't have padding by default

In [7]:
df.loc[:, "tokenized"] = df["headline"].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))

max_length = max(len(tokens) for tokens in df["tokenized"])

df.loc[:, "padded"] = df["tokenized"].apply(lambda x: x + [tokenizer.pad_token_id] * (max_length - len(x)))


In [8]:
df_subset = df.sample(n=100, random_state=42)  # Random 100 rows
df_subset

,headline,tokenized,padded
121815,Celebrities Emerging From Water Because Hey .....,"[42741, 65, 19491, 48297, 3574, 5638, 4362, 14...","[42741, 65, 19491, 48297, 3574, 5638, 4362, 14..."
76633,Bill Maher Slams The Internet For Killing The ...,"[17798, 38137, 30382, 82, 383, 4455, 1114, 255...","[17798, 38137, 30382, 82, 383, 4455, 1114, 255..."
188697,The Most Expensive NFL Tickets Of The Season: ...,"[464, 4042, 5518, 2021, 5134, 26878, 3226, 383...","[464, 4042, 5518, 2021, 5134, 26878, 3226, 383..."
80361,Students Surprise Starbucks Employee With Gene...,"[28239, 47893, 24527, 36824, 2080, 2980, 516, ...","[28239, 47893, 24527, 36824, 2080, 2980, 516, ..."
65636,Ben Stein: 'I Don't Think Trump Knows A Goddam...,"[11696, 15215, 25, 705, 40, 2094, 470, 11382, ...","[11696, 15215, 25, 705, 40, 2094, 470, 11382, ..."
...,...,...,...
43356,"Trump's ""Extreme Vetting"" Of Refugees Empowers...","[6170, 338, 366, 36716, 569, 35463, 1, 3226, 3...","[6170, 338, 366, 36716, 569, 35463, 1, 3226, 3..."
113206,For $6 You Can Give a Coffee Tree and Help Emp...,"[1890, 720, 21, 921, 1680, 13786, 257, 19443, ...","[1890, 720, 21, 921, 1680, 13786, 257, 19443, ..."
66542,Brad Paisley Debuts A Little Ditty About North...,"[30805, 11243, 271, 1636, 1024, 4360, 82, 317,...","[30805, 11243, 271, 1636, 1024, 4360, 82, 317,..."
91949,Video Shows Man Holding Gun Before Allegedly S...,"[10798, 25156, 1869, 31703, 6748, 7413, 26326,...","[10798, 25156, 1869, 31703, 6748, 7413, 26326,..."


##### Define dataset class

In [9]:
class GPT2Dataset(Dataset):
    def __init__(self, df_subset):
        self.input_ids = torch.tensor(df_subset["padded"].tolist(), dtype=torch.long)
        self.attention_mask = (self.input_ids != tokenizer.pad_token_id).long()  # Mask padding tokens

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
            "labels": self.input_ids[idx],  # GPT-2 is trained using its own inputs as labels
        }

In [10]:
# Train/Validation Split
train_size = int(0.8 * len(df_subset))
train_df, val_df = df_subset[:train_size], df_subset[train_size:]

# Create Dataset
train_dataset = GPT2Dataset(train_df)
val_dataset = GPT2Dataset(val_df)

DistilGPT2 has 6 transformer blocks compared to GPT2, which has 12 transformer blocks. To perform transfer learning properly we will freeze all layers and unfreeze the last 2.

In [11]:
# Load the Pretrained GPT-2 Model with LM head
model = GPT2LMHeadModel.from_pretrained("distilgpt2")  # 50% smaller

# Freeze all layers initially
for param in model.parameters():
    param.requires_grad = False

# Unfreeze last 4 layers
for param in model.transformer.h[-2:].parameters():
    param.requires_grad = True

# Move Model to GPU
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

##### Choose a batch size and num workers

In [12]:
train_loader = DataLoader(train_dataset, batch_size=8, num_workers=1, shuffle=True)  # Reduce num_workers
val_loader = DataLoader(val_dataset, batch_size=8, num_workers=1, shuffle=False)

print("Model and data loaders ready!")

Model and data loaders ready!


In [13]:
logging.basicConfig(level=logging.INFO)

# Set Hugging Face Transformers library to show debug logs
transformers.logging.set_verbosity_debug()

In [14]:
# Print selected device
print(f"🔥 Using device: {device}")

# Print GPU info
if device.type == "cuda":
    print(f"🚀 GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory Allocated: {torch.cuda.memory_allocated(0) / 1024 ** 2:.2f} MB")
    print(f"💾 GPU Memory Reserved: {torch.cuda.memory_reserved(0) / 1024 ** 2:.2f} MB")
    print(f"🔄 CUDA Version: {torch.version.cuda}")
else:
    print("🖥 Running on CPU")


🔥 Using device: cuda
🚀 GPU Name: NVIDIA GeForce RTX 3070 Ti Laptop GPU
💾 GPU Memory Allocated: 319.24 MB
💾 GPU Memory Reserved: 356.00 MB
🔄 CUDA Version: 12.4


Reducing Learning Rate
Why? A high learning rate can cause large weight updates, leading to overwriting GPT-2’s pre-trained knowledge.
How? Use a smaller learning rate than usual when fine-tuning.

In [15]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",  # Where model checkpoints will be saved
    logging_dir="./logs",  # Directory for logging
    logging_strategy="steps",  # Log at every step
    logging_steps=50,  # Log every 50 steps
    report_to=["tensorboard"],  # Log to TensorBoard
    eval_strategy="epoch",  # Evaluate at each epoch
    save_strategy="epoch",  # Save model at each epoch
    save_total_limit=5,  # Keep only last 2 checkpoints
    disable_tqdm=False,  # Enable progress bars
    load_best_model_at_end=True,  # Load best model checkpoint at end
    fp16=True,  # Enable mixed precision for speed
    per_device_train_batch_size=8,  # Adjust batch size to prevent memory issues
    per_device_eval_batch_size=8,  # Same for evaluation
    gradient_accumulation_steps=1,  # Accumulate gradients before updating weights
    learning_rate=5e-5, # Lower than usual (default is 5e-4)
    weight_decay=0.01,  # Prevent drastic weight changes
    num_train_epochs=3
)

PyTorch: setting up devices


In [17]:
# Use Hugging Face Trainer API
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

print(trainer)

Using auto half precision backend


In [18]:
# Measure start time
start_time = time.time()

print("🚀 Training started...")

# Start training
trainer.train()

# Measure end time
end_time = time.time()

Currently training with a batch size of: 8
***** Running training *****
  Num examples = 80
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 30
  Number of trainable parameters = 14,175,744


🚀 Training started...


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,No log,4.462164
2,No log,1.242053
3,No log,0.740644



***** Running Evaluation *****
  Num examples = 20
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-10
Configuration saved in ./results/checkpoint-10/config.json
Configuration saved in ./results/checkpoint-10/generation_config.json
Model weights saved in ./results/checkpoint-10/model.safetensors

***** Running Evaluation *****
  Num examples = 20
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-20
Configuration saved in ./results/checkpoint-20/config.json
Configuration saved in ./results/checkpoint-20/generation_config.json
Model weights saved in ./results/checkpoint-20/model.safetensors

***** Running Evaluation *****
  Num examples = 20
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-30
Configuration saved in ./results/checkpoint-30/config.json
Configuration saved in ./results/checkpoint-30/generation_config.json
Model weights saved in ./results/checkpoint-30/model.safetensors


Training completed. Do not forget to share your model on

In [19]:
training_duration = end_time - start_time
# Print training duration
print(f"✅ Training completed in {training_duration:.2f} seconds ({training_duration/60:.2f} minutes)")

# Print final training state
print(f"📈 Final Epoch: {trainer.state.epoch}")
print(f"📊 Total Training Steps: {trainer.state.global_step}")


✅ Training completed in 23.14 seconds (0.39 minutes)
📈 Final Epoch: 3.0
📊 Total Training Steps: 30


##### Run this command in the terminal for training metrics in tensorboard

`tensorboard --logdir=./logs`

In [20]:
# Save model & tokenizer
model.save_pretrained("../models/gpt2-v1/gpt2_finetuned")
tokenizer.save_pretrained("../models/gpt2-v1/gpt2_finetuned")

print("Model saved successfully!")



Configuration saved in ../models/gpt2-v1/gpt2_finetuned/config.json
Configuration saved in ../models/gpt2-v1/gpt2_finetuned/generation_config.json
Model weights saved in ../models/gpt2-v1/gpt2_finetuned/model.safetensors
tokenizer config file saved in ../models/gpt2-v1/gpt2_finetuned/tokenizer_config.json
Special tokens file saved in ../models/gpt2-v1/gpt2_finetuned/special_tokens_map.json


Model saved successfully!


In [21]:
import torch
import numpy as np
from transformers import Trainer

# Define function to calculate perplexity
def compute_perplexity(eval_loss):
    return np.exp(eval_loss)  # Perplexity = exp(loss)

# Get evaluation loss from trainer
eval_results = trainer.evaluate()
eval_loss = eval_results["eval_loss"]
perplexity = compute_perplexity(eval_loss)

print(f"📝 Evaluation Loss: {eval_loss:.4f}")
print(f"🔢 Perplexity: {perplexity:.4f}")



***** Running Evaluation *****
  Num examples = 20
  Batch size = 8


📝 Evaluation Loss: 0.7406
🔢 Perplexity: 2.0973


In [22]:
from transformers import StoppingCriteria, StoppingCriteriaList

class StopOnWhitespace(StoppingCriteria):
    def __call__(self, input_ids, scores, **kwargs):
        # Stop generation if the last 5 tokens are whitespace
        if all(tokenizer.decode(tok).isspace() for tok in input_ids[0, -5:]):
            return True
        return False

stopping_criteria = StoppingCriteriaList([StopOnWhitespace()])


In [23]:
def generate_text(prompt, top_p=0.9, top_k=50):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    
    output = model.generate(
        input_ids, 
        max_length=100,
        min_length=20,
        do_sample=True,
        num_beams=10,
        length_penalty=1,
        early_stopping=False,
        pad_token_id=tokenizer.eos_token_id,  # Ensure it doesn't stop early due to padding
        temperature=10.1,  # Increase randomness
        top_p=top_p,  # Nucleus sampling (limits probability mass)
        top_k=top_k,  # Only consider the top-k most likely words
        repetition_penalty=2.1,  # Avoid repeated spaces
        stopping_criteria=stopping_criteria
    )
    
    return tokenizer.decode(output[0], skip_special_tokens=True).strip()  # Strip extra spaces

# Test with new settings
examples = [
    "Breaking news:", 
    "Latest update:", 
    "The president announced that",
    "Scientists have discovered a new",
    "Experts warn that",
    "A recent study suggests that",
    "Authorities have confirmed that",
    "In a surprising turn of events,",
    "The stock market responded to",
    "New regulations require companies to",
    "Health officials recommend that",
    "Technology companies are investing in",
    "Sports fans are excited about",
    "The weather forecast predicts",
    "Researchers at MIT have developed",
    "Protests erupted in the city over",
    "The Supreme Court ruled that",
    "Celebrities are reacting to",
    "A new breakthrough in medicine shows that",
    "The United Nations has issued a statement on"
]

for text in examples:
    print(f"📝 Input: {text}")
    print(f"🔮 Output: {generate_text(text)}\n")


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


📝 Input: Breaking news:
🔮 Output: Breaking news: An American pastor who killed three teenage girls in Georgia has urged parents to pray as the church says its message never comes back because 'you want a happy world' MORE (W.Va.), along with other groups seeking prayer that won‡t be seen in the nation for too long, have pulled his story off at an Arlington Christian Community Church last weekend and is urging everyone not to become ‪irrespective of their faith

📝 Input: Latest update:
🔮 Output: Latest update: I added the last message from a previous version by telling you that they have been sent to Google for testing, but only since January 6th in March 2013 - this time.

📝 Input: The president announced that
🔮 Output: The president announced that the campaign was launching a new campaign in support of Hillary Clinton in July — something it has been doing all month and even now

📝 Input: Scientists have discovered a new
🔮 Output: Scientists have discovered a new, nearly identical part

In [24]:
input_ids = tokenizer.encode("Breaking news:", return_tensors="pt").to(device)
print("🔎 Encoded Input:", input_ids)


🔎 Encoded Input: tensor([[29449,  1705,    25]], device='cuda:0')


In [25]:
import torch

def generate_with_token_probabilities(prompt, max_tokens=20):
    """Generate text token by token, displaying each step with probabilities."""
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

    print(f"📝 Starting prompt: {prompt}")
    generated_text = prompt

    for _ in range(max_tokens):
        with torch.no_grad():
            outputs = model(input_ids)
            logits = outputs.logits  # Raw model outputs
        
        # Get probability distribution for next token
        probs = torch.softmax(logits[0, -1, :], dim=-1)  
        sorted_probs, sorted_indices = torch.sort(probs, descending=True)

        # Print top token choices
        print(f"\n🔍 Token step {_+1}")
        for i in range(5):  # Show top 5 predictions
            token = tokenizer.decode([sorted_indices[i].item()])
            print(f"  {token}: {sorted_probs[i].item():.4f}")

        # Select most probable token
        next_token_id = sorted_indices[0].item()
        next_token = tokenizer.decode([next_token_id])

        # Append token to generated text
        generated_text += next_token
        input_ids = torch.cat([input_ids, torch.tensor([[next_token_id]]).to(device)], dim=1)

        # Stop if end token is reached
        if next_token == tokenizer.eos_token:
            print("\n🚫 End of text token reached.")
            break

    print("\n📝 Final generated text:")
    print(generated_text)

# Test the function
generate_with_token_probabilities("The United Nations has issued a statement on")



📝 Starting prompt: The United Nations has issued a statement on

🔍 Token step 1
   the: 0.2629
   Tuesday: 0.0802
   its: 0.0777
   Monday: 0.0730
   Wednesday: 0.0707

🔍 Token step 2
   situation: 0.1036
   issue: 0.0358
   matter: 0.0326
   crisis: 0.0231
   incident: 0.0204

🔍 Token step 3
   in: 0.4491
  .: 0.1002
  ,: 0.0884
   and: 0.0627
   with: 0.0405

🔍 Token step 4
   Syria: 0.3931
   Yemen: 0.0905
   Ukraine: 0.0516
   the: 0.0427
   Gaza: 0.0414

🔍 Token step 5
  ,: 0.3221
  .: 0.2079
   and: 0.0923
  :: 0.0494
   that: 0.0385

🔍 Token step 6
   saying: 0.2135
   calling: 0.1378
   which: 0.0575
   and: 0.0476

🔍 Token step 7
   that: 0.2609
   it: 0.1908
   the: 0.1684
  :: 0.0960
   there: 0.0321

🔍 Token step 8
   the: 0.2347
   it: 0.0978
   ": 0.0672
   there: 0.0434
   a: 0.0280

🔍 Token step 9
   Syrian: 0.1529
   situation: 0.0793
   government: 0.0545
   country: 0.0438
   United: 0.0412

🔍 Token step 10
   government: 0.6223
   army: 0.0511
   regime: 0.0465
   p